In [8]:
import comet_ml

import os
os.environ["COMET_API_KEY"] = "I5AiXfuD0TVuSz5UOtujrUM9i"

PROJECT_NAME="robust-training"
WORKSPACE_NAME="maxheuillet"

# Get all Experiemnts in workspace
api = comet_ml.API()

# all_experiments_in_workspace = api.get_experiments(WORKSPACE_NAME)
all_experiments_in_project = api.get_experiments(WORKSPACE_NAME, PROJECT_NAME)


In [9]:
# Convert the experiment summary to an APIExperiment using the experiment key
from tqdm.notebook import tqdm

data = {}
for i in tqdm( range(len(all_experiments_in_project)) ):
    summary = all_experiments_in_project[i].get_parameters_summary()

    d = {}
    for e in summary:
        if e['name'] in [ 'exp', 'dataset', 'pre_trained', 'run_id', 'arch','aug','sched', 'batch_size','init_lr','sched','pruning_strategy','pruning_ratio','loss_function','iterations']:
            if e['name'] == 'run_id':
                id = e['valueMax']
            else:
                d[e['name']] = e['valueMax']
        
    if id not in data.keys():
        try:
            adv_acc  = all_experiments_in_project[i].get_metrics('final_robust_accuracy')[0]
            d[ 'adv_acc' ] = adv_acc['metricValue']
            nat_acc = all_experiments_in_project[i].get_metrics('final_clean_accuracy')[0]
            d[ 'nat_acc' ] = nat_acc['metricValue']
        except:
            d[ 'adv_acc' ] = None
            d[ 'nat_acc' ] = None

        data[id] = d

data

  0%|          | 0/725 [00:00<?, ?it/s]

{'51329588': {'arch': 'resnet50',
  'batch_size': '1042',
  'dataset': 'CIFAR10s',
  'init_lr': '0.01',
  'iterations': '80',
  'loss_function': 'TRADES_v2',
  'pruning_ratio': '0.3',
  'pruning_strategy': 'decay_based_v2',
  'sched': 'sched',
  'adv_acc': None,
  'nat_acc': None},
 '51329586': {'arch': 'resnet50',
  'batch_size': '1042',
  'dataset': 'CIFAR10s',
  'init_lr': '0.01',
  'iterations': '80',
  'loss_function': 'TRADES_v2',
  'pruning_ratio': '0.7',
  'pruning_strategy': 'score_v1',
  'sched': 'sched',
  'adv_acc': None,
  'nat_acc': None},
 '51329589': {'arch': 'resnet50',
  'batch_size': '1042',
  'dataset': 'CIFAR10s',
  'init_lr': '0.01',
  'iterations': '80',
  'loss_function': 'TRADES_v2',
  'pruning_ratio': '0.3',
  'pruning_strategy': 'TS_pruning',
  'sched': 'sched',
  'adv_acc': None,
  'nat_acc': None},
 '51329595': {'arch': 'resnet50',
  'batch_size': '1042',
  'dataset': 'CIFAR10s',
  'init_lr': '0.01',
  'iterations': '80',
  'loss_function': 'TRADES_v2',
  '

In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Assuming 'data' is already defined or loaded
df = pd.DataFrame(data).T
df['adv_acc'] = pd.to_numeric(df['adv_acc'], errors='coerce')
df['nat_acc'] = pd.to_numeric(df['nat_acc'], errors='coerce')
df['aug'] = df['aug'].fillna('aug')

exp = 'RQ2'
pruning_ratio = ['0.0', '0.3', '0.5', '0.7' ]
dataset = 'CIFAR10'
batch_size = '1042'
aug = 'aug'
sched = 'nosched'
pruning_strategy = 'fixed_random'

init_lr = [  '0.001',  ]
pre_trained = ['imagenet21k_non_robust', 'imagenet1k_non_robust', 'imagenet1k_robust']

# Apply filtering
df = df[(df['dataset'] == dataset) &
        (df['pruning_strategy'] == pruning_strategy) &
        (df['pruning_ratio'].isin(pruning_ratio) ) &
        (df['pre_trained'].isin(pre_trained)) &  # Use .isin() to check if the value is in the list
        (df['batch_size'] == batch_size) &
        (df['aug'] == aug) &
        (df['sched'] == sched) &
        (df['init_lr'].isin(init_lr) )]

# Assuming the dataframe 'df' is already loaded
# Group by all columns except 'adv_acc', 'nat_acc'
group_columns = ['arch', 'batch_size', 'dataset', 'init_lr', 'iterations', 
                 'loss_function', 'pruning_ratio', 'pruning_strategy', 
                 'sched', 'aug', 'pre_trained']

# Group by the relevant columns and calculate the mean of 'adv_acc' and 'nat_acc'
df_avg = df.groupby(group_columns).agg({'adv_acc': 'mean', 'nat_acc': 'mean'}).reset_index()

# Display the averaged DataFrame
df_avg


,arch,batch_size,dataset,init_lr,iterations,loss_function,pruning_ratio,pruning_strategy,sched,aug,pre_trained,adv_acc,nat_acc
0,convnext,1042,CIFAR10,0.001,50,APGD,0.0,fixed_random,nosched,aug,imagenet1k_non_robust,0.4416,0.6856
1,convnext,1042,CIFAR10,0.001,50,APGD,0.3,fixed_random,nosched,aug,imagenet1k_non_robust,0.2810,0.4421
2,convnext,1042,CIFAR10,0.001,50,APGD,0.5,fixed_random,nosched,aug,imagenet1k_non_robust,0.4503,0.6241
3,convnext,1042,CIFAR10,0.001,50,APGD,0.7,fixed_random,nosched,aug,imagenet1k_non_robust,0.4454,0.6201


In [25]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Sort the dataframe by natural and adversarial accuracy
sorted_df = df_avg.sort_values(by=['nat_acc', 'adv_acc'], ascending=[False, False])

# # Identify the Pareto frontier
# pareto_front = []
# current_best = -np.inf

# for index, row in sorted_df.iterrows():
#     if row['adv_acc'] > current_best:
#         pareto_front.append((row['nat_acc'], row['adv_acc'], row['pruning_strategy'], row['pruning_ratio']))
#         current_best = row['adv_acc']

# pareto_front = np.array(pareto_front)

# Color map for different pruning strategies
color_map = {
    'convnext': 'blue',
}

# Dictionary to map pruning_ratio to different marker symbols
marker_map = {
    '0.0': 'circle',
    '0.3': 'square',
    '0.5': 'diamond',
    '0.7': 'cross'
}

# Step 3: Plot using plotly
fig = go.Figure()

# Add all methods as separate scatter points, one for each pruning_strategy
for strategy, color in color_map.items():
    df_subset = df_avg[df_avg['arch'] == strategy]
    
    # For each pruning_ratio, assign a marker
    for pruning_ratio_value, marker in marker_map.items():
        df_pruning_ratio_subset = df_subset[df_subset['pruning_ratio'] == pruning_ratio_value]
        fig.add_trace(go.Scatter(
            x=df_pruning_ratio_subset['nat_acc'],
            y=df_pruning_ratio_subset['adv_acc'],
            mode='markers',
            marker=dict(symbol=marker, color=color, size=10),
            name=f'{strategy} - {pruning_ratio_value}',
            text=df_pruning_ratio_subset['pruning_strategy'],  # Hover text for each point
            showlegend=True
        ))

# Update layout with legend inside the figure and square aspect ratio
fig.update_layout(
    title='Frontier Analysis: ', # + name,
    xaxis_title='Clean Accuracy',
    yaxis_title='Robust Accuracy',
    xaxis=dict(range=[0.4, 0.8]),
    yaxis=dict(range=[0.2, 0.6]),
    margin=dict(l=0, r=0, t=35, b=0),  # Margins set to 0
    width=400,  # Set figure width to 3 inches (300px at 96dpi)
    height=400,  # Set figure height to 3 inches (300px at 96dpi)
    showlegend=True,
    legend=dict(
        x=0.02,  # Position the legend inside the figure
        y=0.98,
        traceorder="normal",
        bgcolor="rgba(255, 255, 255, 1)",  # Semi-transparent background
        bordercolor="Black",
        borderwidth=1
    )
)

# Display the plot
fig.show()

# Save the figure
# fig.write_image("./figures/pruningratio_influence.png")
